In [1]:
library(caret, quiet = TRUE)
library(base64enc)
library(httr, quiet = TRUE)

library(mlbench)


Attaching package: ‘httr’

The following object is masked from ‘package:caret’:

    progress



# Build a Model

In [2]:

## multiclass classification in iris dataset:

data(BostonHousing)
BostonHousing$chas = as.numeric(BostonHousing$chas)

set.seed(1960)

dataset = BostonHousing[, -14] 

create_model  =  function() {

    model <- train(medv ~ ., data = BostonHousing, method = "rf", maxnodes=6)    

    return(model)
}


In [3]:
model = create_model()
# cat(model$feature_names)
# print(model)

In [4]:
pred_labels <- predict(model, BostonHousing[, -14] , type="raw")
df = data.frame(BostonHousing[,14])
names(df) = c("medv")
df$Estimator = pred_labels
df$Error = df$Estimator - df$medv
MAPE = mean(abs(df$Error / df$medv))
summary(df)
MAPE

      medv         Estimator         Error          
 Min.   : 5.00   Min.   :13.31   Min.   :-22.18074  
 1st Qu.:17.02   1st Qu.:17.93   1st Qu.: -1.99554  
 Median :21.20   Median :21.01   Median :  0.35029  
 Mean   :22.53   Mean   :22.49   Mean   : -0.04008  
 3rd Qu.:25.00   3rd Qu.:24.45   3rd Qu.:  2.22286  
 Max.   :50.00   Max.   :43.96   Max.   : 11.06503  

[1] 0.1485856

# SQL Code Generation

In [5]:

test_ws_sql_gen = function(mod) {
    WS_URL = "https://sklearn2sql.herokuapp.com/model"
    WS_URL = "http://localhost:1888/model"
    model_serialized <- serialize(mod, NULL)
    b64_data = base64encode(model_serialized)
    data = list(Name = "xgboost_test_model", SerializedModel = b64_data , SQLDialect = "postgresql" , Mode="caret")
    r = POST(WS_URL, body = data, encode = "json")
    # print(r)
    content = content(r)
    # print(content)
    lSQL = content$model$SQLGenrationResult[[1]]$SQL # content["model"]["SQLGenrationResult"][0]["SQL"]
    return(lSQL);
}

In [6]:
lModelSQL = test_ws_sql_gen(model)
cat(lModelSQL)


WITH "RF_0" AS 
(WITH "DT_node_lookup" AS 
(SELECT "ADS"."KEY" AS "KEY", CASE WHEN ("ADS"."Feature_12" <= 9.545000000000002) THEN CASE WHEN ("ADS"."Feature_5" <= 7.4369999999999985) THEN CASE WHEN ("ADS"."Feature_5" <= 6.803) THEN 7 ELSE 8 END ELSE CASE WHEN ("ADS"."Feature_4" <= 0.6825) THEN 9 ELSE 10 END END ELSE CASE WHEN ("ADS"."Feature_4" <= 0.6635) THEN 5 ELSE 6 END END AS node_id_2 
FROM "INPUT_DATA" AS "ADS"), 
"DT_node_data" AS 
(SELECT "Values".nid AS nid, "Values"."E" AS "E" 
FROM (SELECT 5 AS nid, 19.93867403314917 AS "E" UNION ALL SELECT 6 AS nid, 14.163963963963955 AS "E" UNION ALL SELECT 7 AS nid, 24.83834586466165 AS "E" UNION ALL SELECT 8 AS nid, 32.282692307692315 AS "E" UNION ALL SELECT 9 AS nid, 45.38214285714286 AS "E" UNION ALL SELECT 10 AS nid, 21.9 AS "E") AS "Values"), 
"DT_Output" AS 
(SELECT "DT_node_lookup"."KEY" AS "KEY", "DT_node_lookup".node_id_2 AS node_id_2, "DT_node_data".nid AS nid, "DT_node_data"."E" AS "E" 
FROM "DT_node_lookup" LEFT OUTER JOIN "DT_

# Execute the SQL Code

In [7]:
library(RODBC)
conn = odbcConnect("pgsql", uid="db", pwd="db", case="nochange")
odbcSetAutoCommit(conn , autoCommit = TRUE)

[1] 0

In [8]:
df_sql = dataset
names(df_sql) = sprintf("Feature_%d",0:(ncol(df_sql)-1))
df_sql$KEY = seq.int(nrow(dataset))

sqlDrop(conn , "INPUT_DATA" , errors = FALSE)
sqlSave(conn, df_sql, tablename = "INPUT_DATA", verbose = FALSE)

# df_sql

In [9]:
colnames(df_sql)
# odbcGetInfo(conn)
# sqlTables(conn)

[1] "Feature_0"  "Feature_1"  "Feature_2"  "Feature_3"  "Feature_4" 
 [6] "Feature_5"  "Feature_6"  "Feature_7"  "Feature_8"  "Feature_9" 
[11] "Feature_10" "Feature_11" "Feature_12" "KEY"

In [10]:
df_sql_out = sqlQuery(conn, lModelSQL)
head(df_sql_out[order(df_sql_out$KEY),])

,KEY,Estimator
374,1,26.44055
84,2,22.80415
168,3,34.34079
56,4,34.08041
320,5,32.05006
64,6,25.46612


In [11]:
# df_sql_out

# R RandomForest Output

In [12]:
estimator  =  predict(model, dataset, type = "raw")
df_r_out = data.frame(estimator)
names(df_r_out) = c("Estimator")

df_r_out$KEY = seq.int(nrow(dataset))
head(df_r_out)


Estimator,KEY
26.44055,1
22.80415,2
34.34079,3
34.08041,4
32.05006,5
25.46612,6


# Compare R and SQL output

In [13]:
df_merge = merge(x = df_r_out, y = df_sql_out, by = "KEY", all = TRUE, , suffixes = c("_1","_2"))
head(df_merge)

KEY,Estimator_1,Estimator_2
1,26.44055,26.44055
2,22.80415,22.80415
3,34.34079,34.34079
4,34.08041,34.08041
5,32.05006,32.05006
6,25.46612,25.46612


In [14]:
df_merge$Error = df_merge$Estimator_1 - df_merge$Estimator_2
df_merge$AbsError = abs(df_merge$Error)
head(df_merge)


KEY,Estimator_1,Estimator_2,Error,AbsError
1,26.44055,26.44055,0.000000e+00,0.000000e+00
2,22.80415,22.80415,-3.552714e-15,3.552714e-15
3,34.34079,34.34079,0.000000e+00,0.000000e+00
4,34.08041,34.08041,0.000000e+00,0.000000e+00
5,32.05006,32.05006,0.000000e+00,0.000000e+00
6,25.46612,25.46612,-3.552714e-15,3.552714e-15


In [15]:
df_merge_largest_errors = df_merge[df_merge$AbsError > 0.0001,]
df_merge_largest_errors

,KEY,Estimator_1,Estimator_2,Error,AbsError
23,23,17.92935,17.91996,0.009391431,0.009391431
39,39,21.31817,21.34178,-0.023617533,0.023617533
176,176,25.70728,25.68060,0.026687064,0.026687064
434,434,16.52184,16.50997,0.011866726,0.011866726


In [20]:
nrow(df_merge_largest_errors)
stopifnot(nrow(df_merge_largest_errors) == 0)


[1] 4

ERROR: Error in eval(expr, envir, enclos): nrow(df_merge_largest_errors) == 0 is not TRUE


In [18]:
summary(df_sql_out)

      KEY          Estimator    
 Min.   :  1.0   Min.   :13.31  
 1st Qu.:127.2   1st Qu.:17.93  
 Median :253.5   Median :21.01  
 Mean   :253.5   Mean   :22.49  
 3rd Qu.:379.8   3rd Qu.:24.45  
 Max.   :506.0   Max.   :43.96  

In [17]:
summary(df_r_out)

   Estimator          KEY       
 Min.   :13.31   Min.   :  1.0  
 1st Qu.:17.93   1st Qu.:127.2  
 Median :21.01   Median :253.5  
 Mean   :22.49   Mean   :253.5  
 3rd Qu.:24.45   3rd Qu.:379.8  
 Max.   :43.96   Max.   :506.0  

In [19]:
summary(df_merge)

      KEY         Estimator_1     Estimator_2        Error           
 Min.   :  1.0   Min.   :13.31   Min.   :13.31   Min.   :-2.362e-02  
 1st Qu.:127.2   1st Qu.:17.93   1st Qu.:17.93   1st Qu.: 0.000e+00  
 Median :253.5   Median :21.01   Median :21.01   Median : 0.000e+00  
 Mean   :253.5   Mean   :22.49   Mean   :22.49   Mean   : 4.808e-05  
 3rd Qu.:379.8   3rd Qu.:24.45   3rd Qu.:24.45   3rd Qu.: 0.000e+00  
 Max.   :506.0   Max.   :43.96   Max.   :43.96   Max.   : 2.669e-02  
    AbsError        
 Min.   :0.0000000  
 1st Qu.:0.0000000  
 Median :0.0000000  
 Mean   :0.0001414  
 3rd Qu.:0.0000000  
 Max.   :0.0266871  